In [1]:
import pickle
import numpy as np
import pandas as pd
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader, Subset
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence
from torch import optim
from sklearn.model_selection import KFold
import sklearn
from numpy import random
from model_architecture import MatchPredictor
pd.options.display.max_rows = 50

In [2]:
device = torch.device('cuda' if torch.cuda.is_available else 'cpu')
device

device(type='cuda')

In [9]:
with open('time_series_dataset.pkl', 'rb') as file:
    data = pickle.load(file)

In [ ]:
for values in data.values():
    values[1].drop(['away_name', 'home_name', 'match_date'], inplace=True, axis=1)
    values[2].drop(['away_name', 'home_name', 'match_date'], inplace=True, axis=1)

In [ ]:
for values in data.values():
    print(values[1].info())
    break

<class 'pandas.core.frame.DataFrame'>
Index: 3 entries, 297 to 284
Data columns (total 42 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   away_acc_passes               3 non-null      float64
 1   away_acc_shots                3 non-null      float64
 2   away_corners                  3 non-null      float64
 3   away_excluded_count           3 non-null      float64
 4   away_formation                3 non-null      Int64  
 5   away_fouls                    3 non-null      float64
 6   away_free_kicks               3 non-null      float64
 7   away_goalkeeper_saves         3 non-null      float64
 8   away_goals                    3 non-null      float64
 9   away_inacc_shots              3 non-null      float64
 10  away_mean_raiting             3 non-null      float64
 11  away_offsides                 3 non-null      float64
 12  away_passes                   3 non-null      float64
 13  away_point

In [ ]:
class StatsDataset(Dataset):

    def __init__(self, x, y, kfold=None, curr_fold=None, n_splits=None):
        self.x = x
        self.y = y
        self.curr_fold = curr_fold
        self.kfold = KFold(n_splits=n_splits, shuffle=False) if kfold is not None else None

    def __getitem__(self, index):
        x = self.x[index]
        y = self.y[index]

        static = torch.as_tensor(x[0].values, dtype=torch.float32)
        home = torch.as_tensor(x[1].values, dtype=torch.float32)
        away = torch.as_tensor(x[2].values, dtype=torch.float32)

        target = torch.as_tensor(y, dtype=torch.long)
        return static, home, away, target
    
    def __len__(self):
        return len(self.x)
    
    def get_splits(self):
        if self.kfold is None:
            return None
        
        folds = list(self.kfold.split(self.x))
        train_indices, val_indices = folds[self.curr_fold]

        train_data = self._get_subset(train_indices)
        val_data = self._get_subset(val_indices)

        return train_data, val_data

    def _get_subset(self, indices):
        return Subset(self, indices)

In [ ]:
seasons = [key[2] for key in data.keys()]
max_season = sorted(seasons, reverse=True)[0]

train_seasons = np.arange(1, max_season, 1)
test_seasons = np.random.choice(train_seasons, size=10, replace=False)
train_seasons = np.setdiff1d(train_seasons, test_seasons)

train_data = [value for key, value in data.items() if key[2] in train_seasons]
x_test = [value for key, value in data.items() if key[2] in test_seasons]

y_train = np.array([x[0].pop('result') for x in train_data])
y_test = np.array([x[0].pop('result') for x in x_test])


In [ ]:
model = MatchPredictor(43, 16, 2, 8)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
def collate_fn(batch):
    statics, homes, aways, targets = zip(*batch)

    away_lens = torch.tensor([a.size(0) for a in aways])
    home_lens = torch.tensor([h.size(0) for h in homes])

    home_padded = pad_sequence(homes, batch_first=True, padding_value=0.0)
    away_padded = pad_sequence(aways, batch_first=True, padding_value=0.0)

    home_packed = pack_padded_sequence(home_padded, home_lens, batch_first=True, enforce_sorted=False)
    away_packed = pack_padded_sequence(away_padded, away_lens, batch_first=True, enforce_sorted=False)

    statics = torch.stack(statics)
    targets = torch.stack(targets)

    return {
        'statics' : statics,
        'homes' : (home_packed, home_lens),
        'aways' : (away_packed, away_lens),
        'targets' : targets
    }


In [ ]:
n_splits = 5
batch_size = 64
num_epochs = 1

test_dataset = StatsDataset(x_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=batch_size, num_workers=5, collate_fn=collate_fn)

for epoch in range(num_epochs):
    for fold in range(n_splits):
        train_dataset = StatsDataset(train_data, y_train, kfold=True, curr_fold=fold, n_splits=n_splits)
        train_data, val_data = train_dataset.get_splits()

        train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=False, num_workers=0, collate_fn=collate_fn)
        val_loader = DataLoader(val_data, batch_size=batch_size, num_workers=5, collate_fn=collate_fn)
        print('123456')
        for step, batch in enumerate(train_loader):
            print(step)
            print(batch['statics'], '\n')
            print(batch['homes'], '\n')
            print(batch['aways'], '\n')
            break

123456


TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint64, uint32, uint16, uint8, and bool.